In [ ]:
import pandas as pd
import sqlite3

In [ ]:
# Create a connection
conn = sqlite3.connect('Rumba.db')

# Create a cursor object
cur = conn.cursor()

In [ ]:
# Fetch table names
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# Print table names as a list
for table in tables:
    print(table[0])

In [ ]:
# Load department table into a DataFrame
department = pd.read_sql('SELECT * FROM department', conn)


In [ ]:
# Print dataframe
department


In [ ]:
# Load manager table into a DataFrame
manager = pd.read_sql('SELECT * FROM manager', conn)
manager

In [ ]:
# Query database to get employee table and display data
cur.execute("SELECT * FROM employee")
employees = cur.fetchall()

# Display the rows
print('\nEmployee table content:')
for employee in employees:
    print(employee)

In [ ]:
# Load employee table into a DataFrame
employee_df = pd.read_sql('SELECT * FROM employee', conn)
employee_df

In [ ]:
# Join employee and department table
#Retrieving data
sql = '''SELECT e.employee_id, e.full_name, e.job_role, e.manager_id, d.department_name
        FROM employee e INNER JOIN department d
        ON e.department_id = d.department_id'''

In [ ]:
#Executing the query
cur.execute(sql)

#Fetching all the rows returned by the query
result = cur.fetchall();

In [ ]:
result

In [ ]:
# Load result into a DataFrame
join_table = pd.DataFrame(result, \
                          columns=['Employee_Id','Employee_Name',\
                                   'Job_Role','Manager_Id',\
                                   'Department_Name'])
print('\n The top 5 records of Employee and Department tables are:')
join_table.head()

In [ ]:
# Export the result into a CSV file
join_table.to_csv("emp_dep.csv",index=False)

# Close connection
conn.close()

**Working with SQLAlchemy**

In [ ]:
import sqlalchemy as db

In [ ]:
#Create a connection
engine = db.create_engine('sqlite:///Mydata.db')
conn = engine.connect()

# Create an instance of MetaData
metadata = db.MetaData()

In [ ]:
# Create a table called 'manager'
manager = db.Table('manager', metadata,
db.Column('Id', db.Integer(),primary_key=True),
db.Column('Manager_Name', db.String(255), nullable=False),
db.Column('Title', db.String(255), nullable=False),
db.Column('Gender', db.String(1)))

In [ ]:
# Actaully creates the table
metadata.create_all(engine)

In [ ]:
# Insert multiple rows of data into table
query = db.insert(manager)
values = [{'Id':1, 'Manager_Name':'Jane', 'Title':'General Manager', 'Gender':'F'},
          {'Id':2, 'Manager_Name':'Jacob', 'Title':'Regional Manager', 'Gender':'M'},
          {'Id':3, 'Manager_Name':'Jill', 'Title':'Senior Manager', 'Gender':'F'},
          {'Id':4, 'Manager_Name':'Justin', 'Title':'Branch Manager', 'Gender':'M'},
          {'Id':5, 'Manager_Name':'John', 'Title':'Regional Manager', 'Gender':'M'},
          {'Id':6, 'Manager_Name':'James', 'Title':'Asst. Manager', 'Gender':'F'},
          {'Id':7, 'Manager_Name':'June', 'Title':'Senior Manager', 'Gender':'F'}]
results = conn.execute(query,values)


In [ ]:
# Retrieve table data
output = conn.execute(db.select(manager)).fetchall()
output

In [ ]:
# Commit changes to the database
conn.commit()

# Close connection
conn.close()

In [ ]:
#Create an engine
engine = db.create_engine('sqlite:///Rumba.db')

#Create a connection
conn = engine.connect()
metadata = db.MetaData()

In [ ]:
# Read csv file into a dataframe
df = pd.read_csv('emp_dep.csv')
df.to_sql(con=engine, name="Emp_Dep", if_exists='replace', index=False)
print('Number of records successfully loaded: ', len(df))

In [ ]:
# Call tables object
emp_dep = db.Table('Emp_Dep', metadata, autoload_with=engine)
manager = db.Table('manager', metadata, autoload_with=engine)

In [ ]:
from sqlalchemy import text

# Execute the PRAGMA query to get table info
result = conn.execute(text("PRAGMA table_info(manager)"))

# Print the columns of the manager table
for row in result:
    print(row)


In [ ]:
# Create a join query
join_query = db.select(emp_dep.columns.Employee_Id,\
                       emp_dep.columns.Employee_Name,\
                       emp_dep.columns.Job_Role, \
                       emp_dep.columns.Department_Name, \
                       manager.columns.FirstName ).\
select_from(emp_dep.join(manager,\
                         emp_dep.columns.Manager_Id == manager.columns.Id)).\
order_by(emp_dep.columns.Employee_Id)

# Retrieve data
output = conn.execute(join_query)
result = output.fetchall()

In [ ]:
# Load result into a Dataframe
data = pd.DataFrame(result)
data


In [ ]:

# Close connection
conn.close()